In [1]:
import cv2
from keras.models import load_model
import numpy as np
import math

In [2]:
model = load_model('model-017.model')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [1]:
import cv2
import math

cap = cv2.VideoCapture("maskupdated.mp4")

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

prevs_frame_point = []
count = 0
tracking_object = {}
track_id = 0
while True:
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    
    current_frame_point = []
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    

    for x,y,w,h in faces:
        cx = int((x + x + w)/2)
        cy = int((y + y + w)/2)
        
        current_frame_point.append((cx,cy))
        
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
        
        cv2.circle(frame, (cx,cy), 5,(0,0,255), -1)
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(frame, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2)
    
    
    if count <= 2:
        for pt in current_frame_point:
            for pt2 in prevs_frame_point:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                if distance <30:
                    tracking_object[track_id] = pt
                    track_id += 1
    else:
        tracking_object_copy = tracking_object.copy()
        for object_id,pt2 in tracking_object_copy.items():
            object_exists = False
            for pt in current_frame_point:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
                
                if distance < 30:
                    tracking_object[track_id] = pt
                    object_exists = True
                    continue
            if not object_exists:
                tracking_object.pop()
                
                
    for object_id,pt in tracking_object.items():
        cv2.circle(frame,pt,5,(0,0,255),-1)
        cv2.putText(frame,str(object_id),(pt[0],pt[1] -7),0,1,(0,0,255),2)
        
    cv2.imshow("frame",frame)
    prevs_frame_point = current_frame_point.copy()
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()


NameError: name 'np' is not defined

In [ ]:
import cv2
import math

cap = cv2.VideoCapture("maskupdated.mp4")

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

prevs_frame_point = []
count = 0
tracking_objects = {}
track_id = 0
first_face = []
second_face = []
while True:
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    
    current_frame_point = []
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) != 0:
        cv2.putText(frame,'Persond detected : %s' % len(faces),(30,30),0,1,(0,0,255),2)
        if len(faces) == 2:
            first_face= faces[0]
            second_face = faces[1]
            
            
            fcx = int((first_face[0]+ first_face[0] + first_face[2])/2)
            fcy = int((first_face[1] + first_face[1] + first_face[2])/2)
            scx = int((second_face[0]+ second_face[0] + second_face[2])/2)
            scy = int((second_face[1] + second_face[1] + second_face[2])/2)
            
            dist = math.dist((fcx,fcy),(scx,scy))
            if dist > 180:
                cv2.putText(frame,"Keeping Social Distance",(30,600),0,1,(0,255,0),2)
            else:
                cv2.putText(frame," Not Keeping Social Distance",(30,600),0,1,(0,0,255),2)
        for x,y,w,h in faces:
            cx = int((x + x + w)/2)
            cy = int((y + y + w)/2)
            
            current_frame_point.append((cx,cy))

            face_img=gray[y:y+w,x:x+w]
            resized=cv2.resize(face_img,(100,100))
            normalized=resized/255.0
            reshaped=np.reshape(normalized,(1,100,100,1))
            result=model.predict(reshaped)
            print(result)
            label=np.argmax(result,axis=1)[0]


            cv2.circle(frame, (cx,cy), 5,(0,0,255), -1)
            cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
            cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
            cv2.putText(frame, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2)
        print(cx,cy)
        if count <= 2:
            for pt in current_frame_point:
                for pt2 in prevs_frame_point:
                    distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                    if distance <50:
                        tracking_objects[track_id] = pt
                        track_id += 1
        else:
            tracking_objects_copy = tracking_objects.copy()
            current_frame_point_copy = current_frame_point.copy()
            for object_id, pt2 in tracking_objects_copy.items():
                object_exists = False
                for pt in current_frame_point_copy:
                    distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
                    # Update IDs position
                    if distance < 50:
                        tracking_objects[object_id] = pt
                        object_exists = True
                        if pt in current_frame_point:
                            current_frame_point.remove(pt)
                        continue
                # Remove IDs lost
                if not object_exists:
                    tracking_objects.pop(object_id)
            # Add new IDs found
            for pt in current_frame_point:
                tracking_objects[track_id] = pt
                track_id += 1


        for object_id,pt in tracking_objects.items():
            cv2.circle(frame,pt,5,(0,0,255),-1)
            cv2.putText(frame,str(object_id),(pt[0],pt[1] -7),0,1,(0,0,255),2)
    else:
        cv2.putText(frame,"NO person Detected",(30,30),0,1,(0,0,255),2)
    
    
    cv2.imshow("frame",frame)
    prevs_frame_point = current_frame_point.copy()
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 33ms/step
[[0.95924795 0.04075201]]
1/1 [==============================] - 0s 47ms/step
[[0.01071289 0.98928714]]
142 353
1/1 [==============================] - 0s 43ms/step
[[0.95949966 0.0405004 ]]
1/1 [==============================] - 0s 44ms/step
[[0.01073664 0.98926336]]
142 353
1/1 [==============================] - 0s 47ms/step
[[0.01300976 0.9869903 ]]
1/1 [==============================] - 0s 47ms/step
[[0.9655668  0.03443317]]
378 386
1/1 [==============================] - 0s 45ms/step
[[0.01461943 0.9853806 ]]
1/1 [==============================] - 0s 44ms/step
[[0.9655505  0.03444951]]
378 386
1/1 [==============================] - 0s 49ms/step
[[0.01274166 0.9872584 ]]
1/1 [==============================] - 0s 46ms/step
[[0.9655525  0.03444756]]
378 386
1/1 [==============================] - 0s 39ms/step
[[0.9655525  0.03444756]]
1/1 [==============================] - 0s 47ms/step
[[0.01345522 0.9865448 ]]
144 352
1/1 [===========

1/1 [==============================] - 0s 29ms/step
[[0.00819959 0.99180037]]
1/1 [==============================] - 0s 44ms/step
[[0.845416   0.15458407]]
377 385
1/1 [==============================] - 0s 46ms/step
[[0.845416   0.15458407]]
1/1 [==============================] - 0s 45ms/step
[[0.00817494 0.991825  ]]
141 356
1/1 [==============================] - 0s 43ms/step
[[0.00951372 0.9904863 ]]
1/1 [==============================] - 0s 49ms/step
[[0.845416   0.15458407]]
377 385
1/1 [==============================] - 0s 42ms/step
[[0.845416   0.15458407]]
1/1 [==============================] - 0s 46ms/step
[[0.01160176 0.98839825]]
138 358
1/1 [==============================] - 0s 43ms/step
[[0.845416   0.15458407]]
1/1 [==============================] - 0s 46ms/step
[[0.01457966 0.9854203 ]]
139 358
1/1 [==============================] - 0s 41ms/step
[[0.845416   0.15458407]]
1/1 [==============================] - 0s 51ms/step
[[0.01111827 0.98888177]]
139 358
1/1 [===========

1/1 [==============================] - 0s 36ms/step
[[0.0145352  0.98546475]]
1/1 [==============================] - 0s 52ms/step
[[0.9122183  0.08778172]]
377 385
1/1 [==============================] - 0s 38ms/step
[[0.01606575 0.9839342 ]]
1/1 [==============================] - 0s 42ms/step
[[0.9122571  0.08774287]]
377 385
1/1 [==============================] - 0s 41ms/step
[[0.01714264 0.98285735]]
1/1 [==============================] - 0s 44ms/step
[[0.9122571  0.08774287]]
377 385
1/1 [==============================] - 0s 38ms/step
[[0.01653201 0.98346794]]
1/1 [==============================] - 0s 44ms/step
[[0.9122571  0.08774287]]
377 385
1/1 [==============================] - 0s 36ms/step
[[0.01358673 0.98641324]]
1/1 [==============================] - 0s 48ms/step
[[0.9122571  0.08774287]]
377 385
1/1 [==============================] - 0s 32ms/step
[[0.01367132 0.98632866]]
1/1 [==============================] - 0s 50ms/step
[[0.9122571  0.08774287]]
377 385
1/1 [===========

1/1 [==============================] - 0s 42ms/step
[[0.01427772 0.9857223 ]]
1/1 [==============================] - 0s 46ms/step
[[0.9122675  0.08773246]]
377 385
1/1 [==============================] - 0s 41ms/step
[[0.01258965 0.98741037]]
1/1 [==============================] - 0s 46ms/step
[[0.91936713 0.08063287]]
377 386
1/1 [==============================] - 0s 42ms/step
[[0.0153944  0.98460555]]
1/1 [==============================] - 0s 46ms/step
[[0.9196806  0.08031942]]
377 386
1/1 [==============================] - 0s 42ms/step
[[0.01697165 0.9830284 ]]
1/1 [==============================] - 0s 46ms/step
[[0.9195748  0.08042519]]
377 386
1/1 [==============================] - 0s 45ms/step
[[0.01544494 0.98455507]]
1/1 [==============================] - 0s 47ms/step
[[0.9195818  0.08041824]]
377 386
1/1 [==============================] - 0s 42ms/step
[[0.01479921 0.98520076]]
1/1 [==============================] - 0s 47ms/step
[[0.9195737  0.08042632]]
377 386
1/1 [===========